# Tutorial 1

In [ ]:
import numpy as np
import gmsh
import sys

# Initialize the Gmsh API
gmsh.initialize()
gmsh.model.list()

In [ ]:
gmsh.model.add("t1")

lc = 1e-2   

# coordinates, meshsize, tag
gmsh.model.geo.addPoint(0,   0, 0, lc, 1)
gmsh.model.geo.addPoint(.1,  0, 0, lc, 2)
gmsh.model.geo.addPoint(.1, .3, 0, lc, 3)
p4 = gmsh.model.geo.addPoint(0, .3, 0, lc)

gmsh.model.geo.addLine(1,2,1)
gmsh.model.geo.addLine(3,2,2)
gmsh.model.geo.addLine(3,p4,3)
gmsh.model.geo.addLine(4,1,p4)

# list of points, tag
gmsh.model.geo.addCurveLoop([4,1,-2,3],1)


In [3]:
# list of curvee loops, tag
gmsh.model.geo.addPlaneSurface([1],1)
gmsh.model.geo.synchronize()


In [ ]:
gmsh.model

In [ ]:
# dimension, list, tag
gmsh.model.addPhysicalGroup(1, [1,2,4], 5)
gmsh.model.addPhysicalGroup(2, [1], name="my surface")

In [5]:
gmsh.model.list()
# to be sure you are still on the same model
gmsh.model.setCurrent('t1')

In [ ]:

# get the list of all 2D entities (dim)
s = gmsh.model.getEntities(1)
s

In [ ]:
physical_groups = gmsh.model.getPhysicalGroups()
physical_groups

In [ ]:
# get the boundary of the entity (dim,tag) - combined, oriented, recursive (up to dim 0)
b = gmsh.model.getBoundary([(2,1)], True, True, False)
b

In [ ]:
# adiacent entities (dim,tag) in the dim direction (superior dim and inferior dim)
u,d = gmsh.model.getAdjacencies(2,1)
u,d

In [ ]:
# get the list of entities in the bounding box (xmin,ymin,zmin,xmax,ymax,zmax,dim), no dim means all
gmsh.model.getEntitiesInBoundingBox(0.0,0.0,0.0,0.1,0.3,0.0,2)

In [ ]:
# get the dimension of the model, that is, the maximum dimension of the entities
gmsh.model.getDimension()

In [12]:
if False:
    # remove a list of entities; if recursive is True, all the entities that depend on the entities in the list are removed
    gmsh.model.removeEntities([(2,1)],False)
    gmsh.model.getEntities(1),gmsh.model.getEntities(2)

In [ ]:
# get the type of an entity
gmsh.model.getType(1,1)

In [ ]:
# parent of a partitioned model
gmsh.model.getParent(1,1)

In [ ]:
# get the coordinate of the mid-point of a curve
lb,ub = gmsh.model.getParametrizationBounds(1, 1)
lb[0],ub[0]


In [ ]:
# get the point on an entity that is closet to a given point
gmsh.model.getClosestPoint(1,1,[.1,2.0,0.])

In [ ]:
# get the coordinate of the point at the given parametrization location
coord = gmsh.model.getValue(1,1,[(lb[0]+ub[0])/2])
coord 

In [ ]:
# get the parametrization location of the given point
param = gmsh.model.getParametrization(1,1,coord)
param

In [ ]:
# the third argument is a list of coordinates
gmsh.model.getDerivative(1,1,[(lb[0]+ub[0])/2])


In [ ]:
# the third argument is a list of coordinates
gmsh.model.getSecondDerivative(1,1,[(lb[0]+ub[0])/2])

In [ ]:
# the third argument is a list of coordinates
gmsh.model.getCurvature(1,1,[(lb[0]+ub[0])/2])

In [ ]:
# set the visibility of a list of entities recursively
gmsh.model.setVisibility([(1,1)],1,True)
gmsh.model.getVisibility(1,1)

In [23]:
gmsh.model.setColor([(1,1)],0,255,255,255,True)

In [31]:
attrs = gmsh.model.getAttributeNames()
for attr in attrs:
    print(attr)
    print(gmsh.model.getAttribute(1,1,attr))

In [34]:
# generate a 2d mesh
gmsh.model.mesh.generate(2)

In [38]:
# partitione the mesh in 4 groups
gmsh.model.mesh.partition(4)

In [42]:
# visualize the model, whatever you do here it is avaialble later in the script
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()


In [43]:
gmsh.write("t1.msh")
if False:
    gmsh.finalize()

# Tutorial 2: meshing of solids
Here I mesh a step file using the occ module (feature like group-based selection of entities does not work)

In [ ]:
import numpy as np
import gmsh
import sys
import os
import math

# Initialize the Gmsh API
gmsh.initialize()
gmsh.model.add("t20")
gmsh.model.list()

In [2]:
v = gmsh.model.occ.import_shapes("two_blocks.stp")

# remove matching entities
gmsh.model.occ.remove_all_duplicates()

# Finally, let's specify a global mesh size and mesh the partitioned model:
gmsh.option.set_number("Mesh.MeshSizeMin", 0.1)
gmsh.option.set_number("Mesh.MeshSizeMax", 0.2)

In [25]:
p = gmsh.model.get_entities(1)

In [ ]:
gmsh.model.occ.get_surface_loops(1),gmsh.model.occ.get_surface_loops(2)

In [ ]:
gmsh.model.occ.get_matrix_of_inertia(3,2),gmsh.model.occ.get_matrix_of_inertia(3,1)

In [ ]:
gmsh.model.occ.get_mass(3,2),gmsh.model.occ.get_mass(3,1)

In [ ]:
gmsh.model.occ.get_center_of_mass(3,2),gmsh.model.occ.get_center_of_mass(3,1)

In [ ]:
gmsh.model.add_physical_group(3, [1],name="my concave volume")
gmsh.model.add_physical_group(3, [2],name="my cube")

In [ ]:
gmsh.model.add_physical_group(2, [1,2,3],name="my surfaces")

In [ ]:

gmsh.model.occ.synchronize()

gmsh.model.get_boundary([(3,1)], True, True, False),gmsh.model.get_boundary([(3,2)], True, True, False)

In [ ]:
gmsh.model.get_physical_groups(-1)

In [ ]:
gmsh.model.occ.mesh.set_size(p, 0.01)

# this is to get structured meshes in 3d ... not clear how
#gmsh.model.mesh.set_transfinite_volume(1,[])
#gmsh.model.mesh.set_transfinite_volume(2,[])

#gmsh.model.occ.synchronize()
#gmsh.model.mesh.clear()

# Finally, let's specify a global mesh size and mesh the partitioned model:
gmsh.option.set_number("Mesh.MeshSizeMin", 0.1)
gmsh.option.set_number("Mesh.MeshSizeMax", 0.2)

gmsh.model.get_entities(2)

In [ ]:
# Meshing algorithms can changed globally using options:
#gmsh.option.set_number("Mesh.Algorithm", 3)  # Frontal-Delaunay for 2D meshes

# They can also be set for individual surfaces, e.g. for using `MeshAdapt' on
# surface 1:
#gmsh.model.mesh.set_algorithm(2,1,1)

gmsh.model.mesh.generate(2)

# get the nodes for the physical group (does not work with occ)
nodeTags,nodeCoords = gmsh.model.mesh.get_nodes_for_physical_group(2,3)
nodeTags,nodeCoords

In [70]:


# visualize the model, whatever you do here it is avaialble later in the script
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()


# Tutorial 3: meshing of solids
Here I mesh a step file using the geo module (it seems that it works much better than occ).

In [1]:
import numpy as np
import gmsh
import sys
import os
import math

# Initialize the Gmsh API
gmsh.initialize()
gmsh.model.add("t20")
gmsh.merge("two_blocks.stp")

In [5]:
# remove matching entities
gmsh.model.geo.remove_all_duplicates()

In [3]:
gmsh.model.add_physical_group(3, [1],name="my concave volume")
gmsh.model.add_physical_group(3, [2],name="my cube")

2

In [4]:
gmsh.model.get_boundary([(3,1)], True, True, False),gmsh.model.get_boundary([(3,2)], True, True, False)

([(2, -1), (2, -2), (2, -3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, -9)],
 [(2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, -15)])

In [6]:
gmsh.model.get_entities(3)

[(3, 1), (3, 2)]

In [7]:
gmsh.model.get_boundary([(3,1)], True, True, False),gmsh.model.get_boundary([(3,2)], True, True, False)

([(2, -1), (2, -2), (2, -3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, -9)],
 [(2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, -15)])

In [8]:
# Finally, let's specify a global mesh size and mesh the partitioned model:
gmsh.option.set_number("Mesh.MeshSizeMin", 0.1)
gmsh.option.set_number("Mesh.MeshSizeMax", 0.2)

gmsh.model.get_physical_groups(-1)

[(3, 1), (3, 2)]

In [9]:
gmsh.model.mesh.generate(3)

In [10]:

# get the nodes for the physical group (does not work with occ)
nodeTags,nodeCoords = gmsh.model.mesh.get_nodes_for_physical_group(3,2)
nodeTags,nodeCoords

(array([    15,     16,     17, ..., 125858, 125859, 125860], dtype=uint64),
 array([ -5.        ,   6.        , -10.        , ...,  -6.26307658,
         11.89122527,  -2.81773515]))

In [11]:
gmsh.model.geo.synchronize()

# visualize the model, whatever you do here it is avaialble later in the script
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()